# Router

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

prompt_infos = [
    {
        "name": "restaurants",
        "description": "Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template,
    },
    {
        "name": "transport",
        "description": "Good for guiding the transport to get the place",
        "prompt_template": transport_template,
    },
    {
        "name": "destination",
        "description": "Good for recommending place to tour",
        "prompt_template": destination_template,
    },
]

destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = prompt | model | StrOutputParser()
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = PromptTemplate.from_template("{input}")
default_chain = default_prompt | model | StrOutputParser()

router_prompt = PromptTemplate(
    template="""
For the given question, refer to the explanations for the following categories below and select the appropriate category. 
If there is no suitable category, return the string "default".
The output should be single word 

Categories are composed of strings in the format "Category: Description", and the categories are as follows:
{destinations_str}

Question:
{input}
""",
    input_variables=["input"],
    partial_variables={"destinations_str": destinations_str},
)

print(destinations_str)

router_chain = (
    {"input": itemgetter("input")} | router_prompt | model | StrOutputParser()
)

branch = RunnableBranch(
    *(
        (
            lambda x, name=prompt_info["name"], chain=destination_chains[name]: name
            in x["destination"].replace("\n", ""),
            chain,
        )
        for prompt_info in prompt_infos
    ),
    default_chain,
)

full_chain = {"input": itemgetter("input"), "destination": router_chain} | branch

input = "What is best restaurant in Seoul?"
output = full_chain.invoke({"input": input})

print(output)

restaurants: Good for recommending restaurants around the tourist destinations
transport: Good for guiding the transport to get the place
destination: Good for recommending place to tour
Seoul is known for its vibrant food scene, offering a wide range of dining options. Here are a few highly recommended restaurants that showcase the best of Korean cuisine:

1. **Gwangjang Market**: While not a single restaurant, this bustling market is famous for its street food. Try the bindaetteok (mung bean pancakes) and bibimbap.

2. **Myeongdong Kyoja**: Known for its delicious handmade noodles and dumplings, this restaurant is a must-visit for a taste of traditional Korean comfort food.

3. **Jinjujip**: Famous for its kalguksu (knife-cut noodles) and mandu (dumplings), this restaurant offers a cozy atmosphere and hearty meals.

4. **Tosokchon Samgyetang**: Renowned for its ginseng chicken soup, this restaurant is a favorite among locals and tourists alike, especially during the colder months.

5